In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../Data Extraction/10subs_500.csv')

In [47]:
df.head()

,comment,subreddit,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,>I CAN’T BREATHE!\r\n\r\nNEITHER CAN SHE! LOL,the_donald,0.146345,0.007168,0.047354,0.003085,0.050958,0.009246
1,She will never go away.,the_donald,0.171063,0.009457,0.053309,0.006602,0.054556,0.010904
2,> I’d Like to Be President\r\n\r\nHahah. we k...,the_donald,0.089804,0.006587,0.032664,0.003291,0.043775,0.010575
3,Look at me. Do it. Do it.,the_donald,0.076816,0.006302,0.027850,0.003069,0.022476,0.007098
4,"""I think I can rig it better this time""\r\n\r\n",the_donald,0.028194,0.004102,0.021048,0.002638,0.014183,0.004939


In [33]:
classifiers = []

In [6]:
from sklearn.externals import joblib

In [34]:
for i in range(6):
    classifiers.append(joblib.load("lr_char_" + str(i) + ".joblib"))

In [30]:
train = pd.read_csv('C:/Users/anees/Documents/CS/MP/datasets/toxic comment classification/train.csv')
test = pd.read_csv('C:/Users/anees/Documents/CS/MP/datasets/toxic comment classification/test.csv')

In [12]:
all_text = pd.concat([train.comment_text, test.comment_text])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [15]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

In [16]:
word_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)

In [18]:
char_vectorizer.fit(all_text)

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50000, min_df=1,
        ngram_range=(2, 6), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents='unicode', sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [20]:
df_wordfeat = word_vectorizer.transform(df.comment)

In [22]:
df_charfeat = char_vectorizer.transform(df.comment)

In [24]:
df_feat = hstack([df_charfeat,df_wordfeat])

In [23]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [27]:
df['toxic'] = '0'
df['severe_toxic'] = '0'
df['obscene'] = '0'
df['threat'] = '0'
df['insult'] = '0'
df['identity_hate'] = '0'

In [36]:
i = 0
for class_name in class_names:
    df[class_name] = classifiers[i].predict_proba(df_feat)[:,1]
    i+=1

In [63]:
pd.unique(df['subreddit'])

array(['the_donald', 'news', 'politics', 'conservative', 'gaming',
       'Showerthoughts', 'science', 'movies', 'wholesomememes',
       'todayilearned'], dtype=object)

In [59]:
subs = pd.unique(df['subreddit'])
toxic_scores = []
for sub in subs:
    score = []
    for c in class_names:
        x = df[df['subreddit']==sub]
        score.append(x[c].sum())
    toxic_scores.append(score)

In [64]:
for i in range(len(subs)):
    print(subs[i])
    print(toxic_scores[i])

the_donald
[67.41597157978038, 4.667236962577098, 33.8658077359903, 1.8400864226076912, 27.144472900884388, 4.847290443130764]
news
[70.65648165423084, 4.824101765019295, 38.293071678369, 1.5957025245185072, 27.51395654811287, 5.194907154092444]
politics
[70.15577638932052, 4.8877266305598495, 35.431173394393035, 1.5980311829035454, 27.977736740309723, 5.261650786550357]
conservative
[49.28997215477165, 3.1561510461828592, 21.2401455506986, 1.5066688660711445, 18.124962644994802, 5.013339820926488]
gaming
[78.51575594901911, 5.230822205604779, 38.753472987389316, 2.0749716389293953, 30.24755450092626, 5.927758254258103]
Showerthoughts
[70.76612107433485, 4.742585531128775, 33.12862368497479, 1.850230354248961, 29.828658699749617, 5.115507969036546]
science
[38.5452624432364, 3.394550721149722, 16.79017613933849, 1.411108031414023, 15.586134943383051, 3.9582799164115228]
movies
[55.049187985316536, 3.973082855056554, 30.46428318972195, 1.6203676990799494, 20.30895961497749, 4.4977399143

In [58]:
x.head()

,comment,subreddit,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,>I CAN’T BREATHE!\r\n\r\nNEITHER CAN SHE! LOL,the_donald,0.146345,0.007168,0.047354,0.003085,0.050958,0.009246
1,She will never go away.,the_donald,0.171063,0.009457,0.053309,0.006602,0.054556,0.010904
2,> I’d Like to Be President\r\n\r\nHahah. we k...,the_donald,0.089804,0.006587,0.032664,0.003291,0.043775,0.010575
3,Look at me. Do it. Do it.,the_donald,0.076816,0.006302,0.027850,0.003069,0.022476,0.007098
4,"""I think I can rig it better this time""\r\n\r\n",the_donald,0.028194,0.004102,0.021048,0.002638,0.014183,0.004939
